In [ ]:
%pip install duckdb

In [16]:
import duckdb

conn = duckdb.connect()

In [2]:
conn = duckdb.connect(database = 'mydb.duckdb', read_only = False)

In [4]:
import duckdb

conn = duckdb.connect()
conn.execute('''
  CREATE TABLE flights 
    as 
  SELECT 
    * 
  FROM read_csv_auto('./datasets/flights/flights_sample.csv')    
''')

In [ ]:
import duckdb
conn = duckdb.connect()
conn.execute('''
  CREATE OR REPLACE TABLE flights 
    as 
  FROM './datasets/flights/flights_sample.csv'
''').df()

,Count
0,1000


In [1]:
import duckdb
# flights.csv broken into 12 parquet files by year and month as GH does not allow large files in repo
# import all 12 parquet files in a directory using wildcard *
# ./datasets/flights/year=2015/month=??/flights.parquet
conn = duckdb.connect()
conn.execute('''
  CREATE TABLE flights 
    as 
  SELECT
    *
  FROM read_parquet('./datasets/flights/year=2015/month=??/flights.parquet')
             
''').df()

,Count
0,5819079


In [ ]:
import duckdb
# flights.csv broken into 12 parquet files by year and month as GH does not allow large files in repo
# import all 12 parquet files in a directory using wildcard *
# ./datasets/flights/year=2015/month=??/flights.parquet
conn = duckdb.connect()
conn.execute('''
  CREATE TABLE flights 
    as 
  SELECT
    *
  FROM './datasets/flights/year=2015/month=??/flights.parquet' (FORMAT PARQUET)
             
''').df()

In [2]:
display(conn.execute('SHOW TABLES').df())

,name
0,flights


In [ ]:
display(conn.execute('SELECT * FROM flights LIMIT 1000').df())

: 

In [3]:
conn.execute('''
  CREATE TABLE airports(
    IATA_CODE VARCHAR, AIRPORT VARCHAR, CITY VARCHAR,
    STATE VARCHAR, COUNTRY VARCHAR, LATITUDE VARCHAR,
    LONGITUDE VARCHAR); 
  COPY airports FROM './datasets/flights/airports.csv' (AUTO_DETECT TRUE);
''')

display(conn.execute('SELECT * FROM airports').df())

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.6884,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [4]:
result = conn.execute('''
  SELECT COUNT(*) AS column_count
  FROM information_schema.columns
  WHERE table_name = 'airports';
''').fetchall()

# Print the number of columns
print(f"Number of columns: {result[0][0]}")

Number of columns: 7


In [5]:
conn.execute('''
  DROP TABLE IF EXISTS airports;
  CREATE TABLE airports
  AS
  FROM read_csv('./datasets/flights/airports.csv', all_varchar=true)
''')

In [6]:
display(conn.execute('SHOW TABLES').df())

,name
0,airports
1,flights


In [7]:
airlines = conn.execute('''
  SELECT 
    * 
  FROM read_csv('./datasets/flights/airlines.csv',  
                Header = True, 
                Columns = {'IATA_CODE': 'VARCHAR', 'AIRLINE': 'VARCHAR'})
''').df()

airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [8]:
airlines = conn.execute('''
  SELECT 
    * 
  FROM read_csv_auto('./datasets/flights/airlines.csv')
''').df()
airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [9]:
conn.register("airlines", airlines)

In [10]:
display(conn.execute('SHOW TABLES').df())

,name
0,airlines
1,airports
2,flights


In [11]:
display(conn.execute('SELECT * FROM airlines').df())

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [ ]:
import pandas as pd

# load the CSV using pandas
df_airlines = pd.read_csv("./datasets/flights/airlines.csv")

# associate the dataframe with the DuckDB database
conn.register("airlines", df_airlines)

In [12]:
conn.execute('''
COPY
  (SELECT IATA_CODE, LATITUDE, LONGITUDE FROM airports)
TO
  './datasets/flights/airports_location.csv' WITH (HEADER 1, DELIMITER ',');
''')

In [17]:
conn.execute('''
COPY
 (SELECT
   IATA_CODE, LATITUDE, LONGITUDE
  FROM './datasets/flights/airports.csv'
  LIMIT 10)
  TO
    './datasets/flights/airports_location_sample.csv' WITH (HEADER 1, DELIMITER ',');
''')

In [18]:
conn.close()

In [ ]:
%pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fastparquet]


In [19]:
import pandas as pd

df_airports = pd.read_csv("./datasets/flights/airports.csv")
df_airports.to_parquet('./datasets/flights/airports.parquet', engine='fastparquet')

In [20]:
import duckdb

conn = duckdb.connect()
conn.execute('''
  CREATE TABLE airports
    as
  SELECT * FROM read_parquet('./datasets/flights/airports.parquet')
  LIMIT 100
''')

In [21]:
display(conn.execute('SELECT * FROM airports').df())

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
95,EAU,Chippewa Valley Regional Airport,Eau Claire,WI,USA,44.86526,-91.48507
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
97,EGE,Eagle County Regional Airport,Eagle,CO,USA,39.64257,-106.91770
98,EKO,Elko Regional Airport,Elko,NV,USA,40.82493,-115.79170


In [22]:
conn.execute('''
  INSERT INTO airports
  SELECT * FROM read_parquet('./datasets/flights/airports.parquet')
  ORDER BY 1 DESC
  LIMIT 100
''')
display(conn.execute('SELECT * FROM airports').df())

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
195,OME,Nome Airport,Nome,AK,USA,64.51220,-165.44525
196,OMA,Eppley Airfield,Omaha,NE,USA,41.30252,-95.89417
197,OKC,Will Rogers World Airport,Oklahoma City,OK,USA,35.39309,-97.60073
198,OGG,Kahului Airport,Kahului,HI,USA,20.89865,-156.43046


In [23]:
conn.execute('''
  COPY airports
  FROM './datasets/flights/airports.parquet' (FORMAT PARQUET);
''')

In [24]:
conn.execute('''
  COPY airports
  TO
    './datasets/flights/airports_all.parquet' (FORMAT PARQUET);
''')

In [25]:
conn.execute('''
  COPY
    (SELECT * FROM airports LIMIT 100)
  TO
    './datasets/flights/airports_100.parquet' (FORMAT PARQUET);
''')

In [ ]:
# older versions of DuckDB may not have the Excel extension
import duckdb

conn = duckdb.connect()

conn.execute('INSTALL spatial')
conn.execute('LOAD spatial')
conn.execute('''
  CREATE TABLE airports
  as
  SELECT * FROM st_read('./datasets/flights/airports_and_airlines.xlsx', layer='airports');
''')

display(conn.execute('SELECT * FROM airports').df())

In [28]:
# newer versions of DuckDB have the Excel extension
import duckdb

conn = duckdb.connect()

conn.execute('INSTALL excel')
conn.execute('LOAD excel')
conn.execute('''
  CREATE TABLE airports
  as
  SELECT * FROM read_xlsx('./datasets/flights/airports_and_airlines.xlsx', sheet='airports');
''')

display(conn.execute('SELECT * FROM airports').df())

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [ ]:
import os
os.environ['OGR_XLSX_HEADERS'] = 'DISABLE'

In [ ]:
os.environ['OGR_XLSX_HEADERS'] = 'FORCE'

In [ ]:
os.environ['OGR_XLSX_HEADERS'] = 'AUTO'

In [ ]:
conn.execute('''
  CREATE TABLE airlines
  AS
  SELECT * FROM read_xlsx('./datasets/flights/airports_and_airlines.xlsx', sheet='airlines');
''')
display(conn.execute('SELECT * FROM airlines').df())

In [29]:
conn = duckdb.connect()

conn.execute('INSTALL excel')
conn.execute('LOAD excel')
conn.execute('''
  CREATE TABLE airlines (
    IATA_CODE STRING,
    AIRLINES STRING
  );
  INSERT INTO airlines
  SELECT * FROM read_xlsx('./datasets/flights/airports_and_airlines.xlsx', sheet='airlines');
''')
display(conn.execute('SELECT * FROM airlines').df())

,IATA_CODE,AIRLINES
0,AA,American Airlines Inc.
1,US,US Airways Inc.
2,F9,Frontier Airlines Inc.
3,B6,JetBlue Airways
4,OO,Skywest Airlines Inc.
5,AS,Alaska Airlines Inc.
6,NK,Spirit Air Lines
7,WN,Southwest Airlines Co.
8,DL,Delta Air Lines Inc.
9,EV,Atlantic Southeast Airlines


In [ ]:
os.environ['OGR_XLSX_FIELD_TYPES'] = 'STRING' # default is AUTO

In [ ]:
# This is needed when running earlier versions of DuckDB without the Excel extension
conn.execute('''
  COPY airlines
  TO './datasets/flights/airlines.xlsx' WITH (FORMAT GDAL, DRIVER 'xlsx');
''')

In [ ]:
# if you have installed DuckDB's Excel extension
# no need to set OGR_XLSX_FIELD_TYPES
conn.execute('''
  COPY airlines
  TO './datasets/flights/airlines.xlsx'
''')

In [31]:
conn.execute('''
    SELECT * FROM read_xlsx('./datasets/flights/airlines.xlsx');
''').df()

,AA,American Airlines Inc.
0,US,US Airways Inc.
1,F9,Frontier Airlines Inc.
2,B6,JetBlue Airways
3,OO,Skywest Airlines Inc.
4,AS,Alaska Airlines Inc.
5,NK,Spirit Air Lines
6,WN,Southwest Airlines Co.
7,DL,Delta Air Lines Inc.
8,EV,Atlantic Southeast Airlines
9,HA,Hawaiian Airlines Inc.


In [ ]:
%pip install mysql-connector-python

In [ ]:
import mysql.connector
import duckdb

# MySQL connection information
mysql_host = 'localhost'
mysql_user = 'user1'
mysql_password = 'password'
mysql_database = 'My_DB'
mysql_table = 'airlines'

# create a DuckDB connection
duckdb_conn = duckdb.connect()

# connect to MySQL
mysql_conn = mysql.connector.connect(
host = mysql_host,
user = mysql_user,
password = mysql_password,
database = mysql_database
)

# create a cursor for MySQL
mysql_cursor = mysql_conn.cursor()

# query data from MySQL
mysql_query = f'SELECT * FROM {mysql_table}'
mysql_cursor.execute(mysql_query)

# create a DuckDB table with the same schema as MySQL
duckdb_create_table_query = \
f'CREATE TABLE airlines (IATA_CODE VARCHAR(2), AIRLINES VARCHAR)'
duckdb_conn.execute(duckdb_create_table_query)

# get column names from MySQL result
mysql_columns = [column[0] for column in mysql_cursor.description]

# fetch data from MySQL and insert into DuckDB table
duckdb_insert_query = \
f'INSERT INTO airlines VALUES ({", ".join(["?" for _ in mysql_columns])})'

for row in mysql_cursor.fetchall():
    duckdb_conn.execute(duckdb_insert_query, row)

# query the data in DuckDB
display(duckdb_conn.execute('SELECT * FROM airlines').df())

# close the MySQL and DuckDB connections
mysql_cursor.close()
mysql_conn.close()
duckdb_conn.close()

In [ ]:
import duckdb

# create a DuckDB connection
conn = duckdb.connect()

# install and load the MySQL extension
conn.execute('INSTALL mysql')
conn.execute('LOAD mysql')

# define MySQL connection parameters
mysql_host     = 'localhost'
mysql_user     = 'user1'
mysql_password = 'password'
mysql_database = 'My_DB'
mysql_table    = 'airlines'
mysql_port     = 3306

# create a MySQL connection
mysql_connection = f'mysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_database}'

attach_command = f'''
    ATTACH 'host={mysql_host} 
    user={mysql_user} 
    password={mysql_password} 
    port={mysql_port} 
    database={mysql_database}' 
    AS mysqldb (TYPE MYSQL);
'''
conn.execute(attach_command)
conn.execute('USE mysqldb;')

display(conn.execute(f'''
    SELECT * FROM {mysql_table}
''').df())

display(conn.execute(f'''
    show tables
''').df())

# close the DuckDB connection
conn.close()